In [1]:
### Collect Samples

In [3]:
import cv2
face_cascade = cv2.CascadeClassifier("face.xml")

def face_extracter(img):
    faces = face_cascade.detectMultiScale(img)
    for x,y,w,h in faces:
        global cropped_face
        cropped_face = img[y:y+h,x:x+w]
    return cropped_face

def collect_samples(person_id):
    cam = cv2.VideoCapture(0)
    count = 0
    while True:
        ret, frame = cam.read()
        if ret == True:
            if face_extracter(frame) is not None:
                count = count + 1
                face = cv2.resize(face_extracter(frame),(300,300))
                file_path = "samples/"+str(person_id)+"/user"+str(count)+".jpg"
                cv2.imwrite(file_path,face)
                cv2.putText(face,str(count),(50,50),cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,1,(0,255,0),2)
                cv2.imshow("Face Cropper",face)
            else:
                print("Face Not Found")
                pass
            if cv2.waitKey(1) == ord('q') or count == 100:
                break
    cam.release()
    cv2.destroyAllWindows()
    print("Collecting Samples Completed")

In [4]:
collect_samples(0)

In [5]:
collect_samples(1)

In [6]:
#### Training the model

In [8]:
import os
import numpy as np
def labels_for_training_data(directory):
    faces = []
    facesId = []
    for path,subdirectory,filenames in os.walk(directory):
        for filename in filenames:
            if filename.startswith("."):
                print("Skipping System File")
                continue
            id = os.path.basename(path)
            img_path = os.path.join(path,filename)
            print("Id",id)
            print("Path",img_path)
            test_img = cv2.imread(img_path)
            if test_img is None:
                print("Image not loaded properly")
                continue
            face_rect,gray_img = faceDetection(test_img)
            if len(face_rect) != 1:
                continue  ## skip in case of multiple person
            (x,y,w,h) = face_rect[0]
            face_image = gray_img[y:y+h,x:x+w]
            faces.append(face_image)
            facesId.append(int(id))
    return faces,facesId

def faceDetection(test_img):
    gray_img = cv2.cvtColor(test_img,cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier("face.xml")
    faces = face_cascade.detectMultiScale(gray_img)
    return faces,gray_img

def train_model(faces,faceId):
    face_model = cv2.face.LBPHFaceRecognizer_create()
    face_model.train(faces,np.array(faceId))
    return face_model;

In [9]:
faces,faceId = labels_for_training_data("samples")
face_model = train_model(faces,faceId)
face_model.write("MyFaceModel.yml")
print("Model Training Success")

Id 0
Path samples\0\user1.jpg
Id 0
Path samples\0\user10.jpg
Id 0
Path samples\0\user100.jpg
Id 0
Path samples\0\user11.jpg
Id 0
Path samples\0\user12.jpg
Id 0
Path samples\0\user13.jpg
Id 0
Path samples\0\user14.jpg
Id 0
Path samples\0\user15.jpg
Id 0
Path samples\0\user16.jpg
Id 0
Path samples\0\user17.jpg
Id 0
Path samples\0\user18.jpg
Id 0
Path samples\0\user19.jpg
Id 0
Path samples\0\user2.jpg
Id 0
Path samples\0\user20.jpg
Id 0
Path samples\0\user21.jpg
Id 0
Path samples\0\user22.jpg
Id 0
Path samples\0\user23.jpg
Id 0
Path samples\0\user24.jpg
Id 0
Path samples\0\user25.jpg
Id 0
Path samples\0\user26.jpg
Id 0
Path samples\0\user27.jpg
Id 0
Path samples\0\user28.jpg
Id 0
Path samples\0\user29.jpg
Id 0
Path samples\0\user3.jpg
Id 0
Path samples\0\user30.jpg
Id 0
Path samples\0\user31.jpg
Id 0
Path samples\0\user32.jpg
Id 0
Path samples\0\user33.jpg
Id 0
Path samples\0\user34.jpg
Id 0
Path samples\0\user35.jpg
Id 0
Path samples\0\user36.jpg
Id 0
Path samples\0\user37.jpg
Id 0
Path 

In [10]:
### Testing

In [11]:
names = {0:"Umang",1:"Rohit"}
cam = cv2.VideoCapture(0)
face_model = cv2.face.LBPHFaceRecognizer_create()
face_model.read("MyFaceModel.yml")
face_cascade = cv2.CascadeClassifier("face.xml")
while True:
    ret,img = cam.read()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    for x,y,w,h in faces:
        single_face = gray[y:y+h,x:x+w]
        id,confidence = face_model.predict(single_face)
        #confidence = 100 - confidence
        print("Id",id)
        print("Confidence",confidence)
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),3)
        predicted_name = names[id]
        if confidence>60:
            cv2.putText(img,predicted_name,(x,y),cv2.FONT_HERSHEY_SCRIPT_SIMPLEX,1,(0,255,0),2)
    cv2.imshow("Webcam",img)
    if cv2.waitKey(1) == ord('q'):
        break
cam.release()
cv2.destroyAllWindows()

Id 1
Confidence 117.95743576263914
Id 1
Confidence 55.11429386717506
Id 1
Confidence 54.956084693286016
Id 1
Confidence 56.6235271735604
Id 1
Confidence 55.831147466719784
Id 1
Confidence 53.606464224488604
Id 1
Confidence 53.64393595292013
Id 1
Confidence 53.64393595292013
Id 1
Confidence 51.31539412562798
Id 0
Confidence 49.16349777229427
Id 0
Confidence 49.358190403351394
Id 0
Confidence 49.358190403351394
Id 0
Confidence 54.34326207465307
Id 0
Confidence 43.59661712058282
Id 0
Confidence 40.93901445413366
Id 0
Confidence 41.88600350523016
Id 0
Confidence 43.04399993132852
Id 0
Confidence 141.61225891592238
Id 0
Confidence 36.636406076715794
Id 0
Confidence 41.67414996499057
Id 0
Confidence 38.942527465352526
Id 0
Confidence 38.06174391454596
Id 0
Confidence 43.46830723965345
Id 0
Confidence 41.220444623489435
Id 0
Confidence 41.06191746799806
Id 0
Confidence 44.40108997371017
Id 0
Confidence 44.662698666312416
Id 0
Confidence 41.30167702872557
Id 0
Confidence 46.684615415079904
Id 

In [12]:
### Car Detection

In [1]:
import cv2

In [9]:
cam = cv2.VideoCapture("video.avi")
car_cascade = cv2.CascadeClassifier("cars.xml")
while True:
    ret, img = cam.read()
    if ret == True:
        cars = car_cascade.detectMultiScale(img)
        for x,y,w,h in cars:
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),3)
            face = img[y:y+h,x:x+w]
        cv2.imshow("Car Detection",img)
        if cv2.waitKey(25) == ord("q"):
            break
    else:
        break
cam.release()
cv2.destroyAllWindows()